### 1 build model, visulization and SCF computation
#### 1.1 Atoms
An Atoms object is a collection of atoms. Here's how to define N2 by directly specifying the positions of two nitrogen atoms molecule by directly specifying the positions of two nitrogen atoms

In [1]:
from ase import Atoms

d=1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

You can also build crystals using the lattice module, which returns Atoms objects corresponding to common crystal structures. Let's make a Cu(111) surface:

In [2]:
from ase.build import fcc111

slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

How does it feel to build a model directly from code? Probably not as solid as the “what you see is what you get” modeling software. Let's take a look at how our model is constructed, which can be done with the view() method of ase.visualize.

In [3]:
from ase.visualize import view

view(molecule)
view(slab) # can be done by vscode

<Popen: returncode: None args: ['/opt/anaconda3/bin/python', '-m', 'ase', 'g...>

view() will bring up an ase.gui window, which is interactive and allows you to change the visualization angle and select atoms, etc.
Note, however, that the default viewer uses the ase.gui window and cannot be displayed in a notebook.
An alternate viewer can be used by specifying the optional keyword viewer=... to use an alternate viewer.

#### 1.2 Adding a Calculator
Most of the calculations in this ASE tutorial use the Effective Medium Theory (EMT) calculator because it is very fast. Note, however, that other calculators can be attached to the ASE module via ase.calculator, such as VASP, GPAW, pyscf, and ABACUS.

we can attach the calculator to a previously created Atoms object:

In [4]:
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()


and use the get_potential_energy() method of the Atoms class to use it to calculate the total energy of the system:

In [5]:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()
print(f'N2 energy:{e_N2}')
print(f'Cu energy:{e_slab}')

N2 energy:0.440343573035614
Cu energy:11.509056283569898


### 2 Structure optimization: the adsorbed state as an example
Let us use the Quasi-Newton optimization (QNO) method to optimize N2 structure of molecules adsorbed on a copper surface. First the adsorbent needs to be added to the copper sheet to construct the surface adsorption model

In [ ]:
from ase.build import add_adsorbate

h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

In order to better simulate the “surface extending out from the bulk phase” system and to speed up the optimization, it is common to calculate the surface adsorption and reaction system by fixing 1-2 layers of atoms relative to the bottom surface.

For simplicity, let's use FixAtoms from the ase.constraints constraints module to fix all Cu atoms, allowing only N2 molecules to relax to the equilibrium structure. Due to the fast calculation speed of the EMT calculator, this atom fixing operation can be left out.

It is important to note that the fixed atoms are simply not involved in the relaxation, not in the calculation. Therefore, the immobilized atom still has an effect on the total energy of the system.

In [6]:
from ase.constraints import FixAtoms

constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

Now connect the proposed Newtonian minimizer to the system and save the trajectory file. Run the optimization calculation at the given convergence criterion, i.e., the forces on all atoms should be less than some fmax:

In [7]:
from ase.optimize import QuasiNewton

dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 17:51:34       11.509056       0.0000


True

Other optimizers can be used here, such as BFGS, CG, etc., which are commonly used in optimization.

In [8]:
from ase.optimize import BFGS
import os
os.system("rm -f N2Cu.traj")

dyn2 = BFGS(slab, trajectory='N2Cu.traj')
dyn2.run(fmax=0.05)

      Step     Time          Energy          fmax
BFGS:    0 17:52:14       11.509056        0.000000


True

By this point we have the total energy of the system after relaxation which is 11.509056.

Let us calculate the adsorption energy.

In [9]:
print('sorption energy:', e_slab + e_N2 - slab.get_potential_energy())

sorption energy: 0.440343573035614


### 3 Input and Output
The input and output of the ASE is done through the methods in the ase.io module.

The write() function can be called to write atomic positions to a file:

In [ ]:
from ase.io import write

write('slab.xyz', slab) # save xyz file

This will write the file in xyz format. The possible formats are:

| format | description |
| ------ | -------------------------- |
| `xyz` | regular xyz format (actually `extxyz` format) |
| `cube` | Gaussian cube file |
| `pdb` | Protein data storage files |
| `traj` | ASE's own trajectory format |
| `py` | Python scripts |

The `extxyz` format here is an enhanced version of the `xyz` format, which retains the basic information of the `xyz` format.
The `extxyz` format is an enhanced version of the `xyz` format, which retains the basic information of the `xyz` format, but adds support for reading and writing various properties of the system (energies, atomic forces, virials, etc.). This format has been used in a variety of software applications, such as the molecular dynamics visualization software Ovito, as well as the graphical neural network machine learning potential Nequip.


The `read()` function of the `ase.io` module is often used to read chemical information from a file, as follows:

In [11]:
from ase.io import read

slab_from_file = read('slab.xyz')

### 4 Case summary: N2 adsorption on Cu
Combining the above codes, it is a continuous code for the whole process of N2 adsorption on the surface of Cu111 simply examined by ASE

In [12]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('sorption energy:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 18:02:54       11.689927       1.0797
BFGSLineSearch:    1[  2] 18:02:54       11.670814       0.4090
BFGSLineSearch:    2[  4] 18:02:54       11.625880       0.0409
sorption energy: 0.32351942231763786


The surface adsorption system obtained from our optimization can also be visualized using the view() function

In [13]:
from ase.visualize import view

view(slab) # can be done by vscode

<Popen: returncode: None args: ['/opt/anaconda3/bin/python', '-m', 'ase', 'g...>

### 5 Example of molecular dynamics calculations
The ase.md module in ase allows for molecular dynamics calculations, so let's run a simple molecular dynamics simulation using the VelocityVerlet algorithm on a nitrogen molecule as an example.

We first create the VelocityVerlet object, assigning it to the molecule (i.e., the Atoms object for N2) and to the time step used for the integration of Newton's laws.

We then execute the dynamics by calling its run() method and telling it how many steps to perform:

In [14]:
from ase.md.verlet import VelocityVerlet
from ase import units # units模块用于定义单位

dyn = VelocityVerlet(molecule, dt=1.0 * units.fs) 

for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=50)

/opt/anaconda3/lib/python3.11/site-packages/ase/md/verlet.py:53: FutureWarning: dt variable is deprecated; please use timestep.
  warnings.warn(


 0: 0.44034 eV, 0.44034 eV, 0.00000 eV
 1: 0.43917 eV, 0.32568 eV, 0.11349 eV
 2: 0.43831 eV, 0.28902 eV, 0.14929 eV
 3: 0.44075 eV, 0.44064 eV, 0.00011 eV
 4: 0.43838 eV, 0.29518 eV, 0.14320 eV
 5: 0.43909 eV, 0.31878 eV, 0.12031 eV
 6: 0.44034 eV, 0.44008 eV, 0.00026 eV
 7: 0.43926 eV, 0.33273 eV, 0.10653 eV
 8: 0.43824 eV, 0.28343 eV, 0.15482 eV
 9: 0.44073 eV, 0.43975 eV, 0.00098 eV


### 6 Summary

In [15]:
# after running the above code, the molecule will move
import os

tmp_files = ['N2Cu.traj', 'slab.xyz']
for i in tmp_files:
    os.remove(i)

In this tutorial, you learned some of the basic approaches in ASE.

Specifically, you learned:

・Define a molecule or crystal using ASE.

・Calculate system energies and perform structural relaxation.

・Read or write atom files using ASE

・Visualize atom files

・Perform molecular dynamics calculations with ASE